In [1]:
!pip -qq install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    EvalPrediction
)
import evaluate
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

import random
import os
import re
import warnings

warnings.filterwarnings("ignore")

2025-11-30 14:39:04.480135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764513544.661600      91 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764513544.714277      91 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
file_path = "/kaggle/input/phishingemails/Phishing_Email.csv"
df_raw = pd.read_csv(file_path, index_col=0).rename(columns={"Email Text": "body", "Email Type": "label_text"})
df_raw["label"] = (df_raw["label_text"] != "Safe Email").astype(int)

df_raw.describe(exclude=int)

,body,label_text
count,18634,18650
unique,17537,2
top,empty,Safe Email
freq,533,11322


In [4]:
df = df_raw[df_raw["body"] != "empty"].dropna(subset="body")

In [5]:
print("Safe Email:\n")
print(df[(df["label"] == 0) & (df["body"].map(len) < 100)].iloc[0, 0])
print("\nPhishing Email:\n")
print(df[(df["label"] == 1) & (df["body"].map(len) < 100)].iloc[0, 0])

Safe Email:

hpl nom for august 15 , 2000 ( see attached file : hplo 815 . xls ) - hplo 815 . xls

Phishing Email:

re [ 11 ] bands leonardo di caprio in 1939 ? ? ? ? ? ? shania twain french hospitals


## Classes and Functionsclass 

In [20]:
class SpecialDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {key: val if torch.is_tensor(val) else torch.tensor(val) for key, val in encodings.items()}
        self.labels = labels if torch.is_tensor(labels) else torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

    def to(self, device):
        self.encodings = {key: val.to(device) for key, val in self.encodings.items()}
        self.labels = self.labels.to(device)
        return self 
    
    
def text_cleaner(text):
    text = text.lower()
    
    # Replace URLs with _url_
    text = re.sub(r'https?://\S+|www\.\S+', '_url_', text)
    
    # Replace emails with _email_
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '_email_', text)
    
    # Remove special characters except ', ", _url_, and _email_
    text = re.sub(r"[^a-zA-Z0-9\s'\"_]", '', text)
    
    # Remove sequences of underscores that are not _url_ or _email_
    def clean_underscores(match):
        text = match.group(0)
        if text in ['_url_', '_email_']:
            return text
        return text.replace('_', '')

    text = re.sub(r'\b_[a-zA-Z0-9_]*_\b', clean_underscores, text)
    
    # Replace sequences of whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading and trailing whitespace
    text = text.strip()
    
    return text

def get_texts_labels(df: pd.DataFrame):
    texts = df["body"].apply(text_cleaner).tolist()
    labels = df["label"].tolist()
    
    return texts, labels

def split_data(texts: list[str], labels: list[str], seed=42):
    train_texts, val_test_texts, train_labels, val_test_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=seed, stratify=labels)

    val_texts, test_texts, val_labels, test_labels = train_test_split(
        val_test_texts, val_test_labels, test_size=0.5, random_state=seed, stratify=val_test_labels)
    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels


def encode_texts(texts: list[str],
                 tokenizer: BertTokenizer,
                 max_length: int = 512):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    return encodings

device = "cuda" if torch.cuda.is_available() else "cpu"
def get_datasets(texts: list[str],
                 labels: list[str],
                 tokenizer_dir: str = '/kaggle/input/huggingface-bert/bert-base-uncased',
                 save: bool = False,
                 move: bool = False):
    global device
    
    tokenizer = BertTokenizer.from_pretrained(tokenizer_dir)

    train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = split_data(texts, labels)

    train_encodings = encode_texts(train_texts, tokenizer)
    val_encodings = encode_texts(val_texts, tokenizer)
    test_encodings = encode_texts(test_texts, tokenizer)
    
    train_dset = SpecialDataset(train_encodings, train_labels)
    val_dset = SpecialDataset(val_encodings, val_labels)
    test_dset = SpecialDataset(test_encodings, test_labels)
    
    if move:
        train_dset = train_dset.to(device)
        val_dset = val_dset.to(device)
        test_dset = test_dset.to(device)
    if save:
        tokenizer.save_pretrained('/kaggle/working/tokenizer_save')
    
    return train_dset, val_dset, test_dset

accuracy_metric = evaluate.load("accuracy")
roc_auc_metric = evaluate.load("roc_auc")

def compute_metrics(p: EvalPrediction):
    global accuracy_metric
    global roc_auc_metric
    preds = np.argmax(p.predictions, axis=1)
    
    accuracy_result = accuracy_metric.compute(predictions=preds, references=p.label_ids)
    
    if p.predictions.shape[1] == 2:
        probs = p.predictions[:, 1]
        roc_auc_result = roc_auc_metric.compute(prediction_scores=probs, references=p.label_ids)
    else:
        roc_auc_result = roc_auc_metric.compute(prediction_scores=p.predictions, references=p.label_ids, multi_class='ovr')

    return {
        "accuracy": accuracy_result["accuracy"],
        "roc_auc": roc_auc_result["roc_auc"]
    }

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
    
def freeze_parameters(model, predicate):
    for name, param in model.named_parameters():
        if predicate(name, param):
          param.requires_grad = False

def unfreeze_parameters(model, predicate):
    for name, param in model.named_parameters():
        if predicate(name, param):
          param.requires_grad = True

def predicate(name, param):
    return not (name.startswith("bert.pooler.dense") or name.startswith("classifier"))

## Creating Datasets

In [8]:
model_path = '/kaggle/input/phishing-email-classifier-bert/transformers/scam-email-classifier-bert-uncased'
tokenizer_path = '/kaggle/input/phishing-email-classifier-bert/transformers/scam-email-bert-tokenizer'
dsets_dir = '/kaggle/input/phishing-email-classifier-bert/phishing-email-encoded-datasets'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

texts, labels = get_texts_labels(df)
print('Creating datasets...')
train_dset, val_dset, test_dset = get_datasets(texts, labels, tokenizer_path, move=True)
print('Datasets created')

print(f'Train size: {len(train_dset)}')
print(f'Validation size: {len(val_dset)}')
print(f'Test size: {len(test_dset)}')

Creating datasets...
Datasets created
Train size: 14480
Validation size: 1810
Test size: 1811


## Using Pre-Trained Model

In [10]:
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    eval_strategy="epoch",
    dataloader_pin_memory=not torch.cuda.is_available(),
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dset,
    eval_dataset=val_dset,
    compute_metrics=compute_metrics
)

results = trainer.predict(test_dset)
metrics = results.metrics

print("Pre-Trained Evaluation Metrics:\n  ")
print("\n  ".join([f"{key}: {val}" for key, val in metrics.items()]))

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Pre-Trained Evaluation Metrics:
  
test_loss: 0.05169203504920006
  test_model_preparation_time: 0.0029
  test_accuracy: 0.9906129210381005
  test_roc_auc: 0.9984881718270916
  test_runtime: 26.5199
  test_samples_per_second: 68.288
  test_steps_per_second: 8.56


## Fine-Tuning model

In [21]:
unfreeze_parameters(model, predicate)
print_trainable_parameters(model)
freeze_parameters(model, predicate)
print_trainable_parameters(model)

trainable params: 109483778 || all params: 109483778 || trainable%: 100.00
trainable params: 592130 || all params: 109483778 || trainable%: 0.54


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Roc Auc
1,0.042800,0.040264,0.002900,0.990055,0.999380
2,0.031100,0.045112,0.002900,0.990055,0.999382
3,0.041500,0.040755,0.002900,0.990608,0.999382


TrainOutput(global_step=5430, training_loss=0.04239851581018494, metrics={'train_runtime': 782.8143, 'train_samples_per_second': 55.492, 'train_steps_per_second': 6.937, 'total_flos': 1.14295442448384e+16, 'train_loss': 0.04239851581018494, 'epoch': 3.0})

In [23]:
results = trainer.predict(test_dset)
metrics = results.metrics

print("Fine-Tuned Evaluation Metrics:\n  ")
print("\n  ".join([f"{key}: {val}" for key, val in metrics.items()]))

Fine-Tuned Evaluation Metrics:
  
test_loss: 0.03856932371854782
  test_model_preparation_time: 0.0029
  test_accuracy: 0.9911651021535064
  test_roc_auc: 0.9985106980025076
  test_runtime: 26.5403
  test_samples_per_second: 68.236
  test_steps_per_second: 8.553


| Metric | Before  | After   |
| ------ | ------- | ------- |
| Loss   | 0.051   | 0.039   |
| Acc    | 0.9906  | 0.9912  |
| ROC-AUC| 0.99849 | 0.99851 |

## ROC-Curve plot and some test dataset examples

In [36]:
preds = np.apply_along_axis(np.argmax, 1, results.predictions)
true_label_0_indices = [i for i, label in enumerate(test_dset.labels.cpu()) if label == 0 and preds[i] == 1]
true_label_1_indices = [i for i, label in enumerate(test_dset.labels.cpu()) if label == 1 and preds[i] == 0]

sample_indices_0 = random.sample(true_label_0_indices, 1)
sample_indices_1 = random.sample(true_label_1_indices, 1)

sample_indices = sample_indices_0 + sample_indices_1

for idx in sample_indices:
    email_encoding = {key: val[idx].unsqueeze(0) for key, val in test_dset.encodings.items()}
    input_ids = email_encoding['input_ids']
    email_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    true_label = test_dset.labels[idx].item()
    predicted_label = np.argmax(results.predictions[idx])

    print(f"Email: {email_text}")
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}\n")

Email: cnet shopper newsletter desktops notebooks editionshopper all cnet the web 1 canon powershot s40 2 canon powershot g2 3 gateway 600xl 4 dell latitude c400 5 cyber shot dscf707 all most popular atlas micro gs 9800 pentium 4 253 ghz 512mb ddr sdram 40gb hard drive cdrwdvdrom 19inch crt nvidia geforce4 ti 4200 just 2099peripherals for your desktop canon s900 photo printerwacom intuis2 6x8 tabletwireless keyboard mouse viewsonic viewpanel vp 181gateway 600xl pentium 4m 18ghz 256mb ram 40gb hard drive 157inch tft active matrix starting at 2899 dell dimension 4000 pentium 4 17ghz to 253ghz 128mb to 1gb up to 120gb hard drive starting at 739 dell inspiron 2600 series 1ghz to 17ghz 128mb to 256mb ram 20gb to 40gb hard drive just 999 hp pavilion n5450 pentium iii 850mhz 128mb ram 20gb hard drive 15inch tft active matrix just 1099 gateway solo 1450se intel celeron 12ghz 128mb ram 20gb hard drive 141inch tft active matrix just 1158 did you know that channelonline's storesite enables you to